In [5]:
import pandas as pd
from IPython.display import display

def find_partial_fd(df, primary_key):
    """
    找出部分函數依賴（Partial Functional Dependencies）
    """
    partial_fd = {}
    non_key_attrs = [col for col in df.columns if col not in primary_key]  # 排除主鍵欄位

    for key in primary_key:
        for attr in non_key_attrs:
            if key in df.columns and attr in df.columns:
                unique_values = df[[key, attr]].drop_duplicates()
                if unique_values[key].nunique() < unique_values[attr].nunique():
                    partial_fd[key] = attr

    return partial_fd

def find_transitive_fd(df, primary_key):
    """
    找出傳遞函數依賴（Transitive Functional Dependencies）
    """
    transitive_fd = {}
    non_key_attrs = [col for col in df.columns if col not in primary_key]  # 排除主鍵

    for attr in non_key_attrs:
        for dep_col in non_key_attrs:
            if attr != dep_col and attr in df.columns and dep_col in df.columns:
                unique_values = df[[attr, dep_col]].drop_duplicates()
                if unique_values[attr].nunique() < unique_values[dep_col].nunique():
                    transitive_fd[attr] = dep_col

    return transitive_fd

def normalize_to_2NF(df, primary_key):
    """
    將 1NF 轉換為 2NF（移除部分函數依賴）
    """
    partial_fd = find_partial_fd(df, primary_key)

    if not partial_fd:
        print("✅ 已經是 2NF")
        return df, {}

    new_tables = {}
    for key, attr in partial_fd.items():
        if attr not in df.columns:
            print(f"⚠️ 警告：欄位 {attr} 不存在於 DataFrame，跳過！")
            continue
        new_table = df[[key, attr]].drop_duplicates()
        new_tables[f"{key}_table"] = new_table
        df = df.drop(columns=[attr], errors='ignore')  # 避免 KeyError

    print("🔄 轉換為 2NF 完成")
    return df, new_tables

def normalize_to_3NF(df, primary_key):
    """
    將 2NF 轉換為 3NF（移除傳遞函數依賴）
    """
    transitive_fd = find_transitive_fd(df, primary_key)

    if not transitive_fd:
        print("✅ 已經是 3NF")
        return df, {}

    new_tables = {}
    for key, attr in transitive_fd.items():
        if attr not in df.columns:
            print(f"⚠️ 警告：欄位 {attr} 不存在於 DataFrame，跳過！")
            continue
        new_table = df[[key, attr]].drop_duplicates()
        new_tables[f"{key}_table"] = new_table
        df = df.drop(columns=[attr], errors='ignore')  # 避免 KeyError

    print("🔄 轉換為 3NF 完成")
    return df, new_tables

# 測試數據
data = {
    'StudID': [1, 2, 3, 1, 2],
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob'],
    'Course': ['Math', 'Science', 'Math', 'English', 'English'],
    'Prof': ['Dr. Lin', 'Dr. Wang', 'Dr. Lin', 'Dr. Smith', 'Dr. Smith'],
    'Location': ['Room 101', 'Room 202', 'Room 101', 'Room 303', 'Room 303'],
    'Score': [90, 85, 88, 75, 80]
}

df_1NF = pd.DataFrame(data)

# 顯示 DataFrame 的函數（適用於 Jupyter Notebook）
def display_dataframe(df, name="DataFrame"):
    """ 在 Jupyter Notebook 中格式化顯示 DataFrame """
    print(f"\n📌 {name}：")
    display(df.style.set_properties(**{'border': '1px solid black', 'padding': '10px'}))

# 顯示原始 1NF 表格
display_dataframe(df_1NF, "原始 1NF 表格")

# 轉換 1NF → 2NF
df_2NF, new_2NF_tables = normalize_to_2NF(df_1NF, ['StudID', 'Course'])

# 顯示 2NF 表格
display_dataframe(df_2NF, "2NF 主要表格")
for name, table in new_2NF_tables.items():
    display_dataframe(table, f"2NF 新表格 {name}")

# 轉換 2NF → 3NF
df_3NF, new_3NF_tables = normalize_to_3NF(df_2NF, ['StudID', 'Course'])

# 顯示 3NF 表格
display_dataframe(df_3NF, "3NF 主要表格")
for name, table in new_3NF_tables.items():
    display_dataframe(table, f"3NF 新表格 {name}")



📌 原始 1NF 表格：


,StudID,Name,Course,Prof,Location,Score
0,1,Alice,Math,Dr. Lin,Room 101,90
1,2,Bob,Science,Dr. Wang,Room 202,85
2,3,Charlie,Math,Dr. Lin,Room 101,88
3,1,Alice,English,Dr. Smith,Room 303,75
4,2,Bob,English,Dr. Smith,Room 303,80


⚠️ 警告：欄位 Score 不存在於 DataFrame，跳過！
🔄 轉換為 2NF 完成

📌 2NF 主要表格：


,StudID,Name,Course,Prof,Location
0,1,Alice,Math,Dr. Lin,Room 101
1,2,Bob,Science,Dr. Wang,Room 202
2,3,Charlie,Math,Dr. Lin,Room 101
3,1,Alice,English,Dr. Smith,Room 303
4,2,Bob,English,Dr. Smith,Room 303



📌 2NF 新表格 StudID_table：


,StudID,Score
0,1,90
1,2,85
2,3,88
3,1,75
4,2,80


✅ 已經是 3NF

📌 3NF 主要表格：


,StudID,Name,Course,Prof,Location
0,1,Alice,Math,Dr. Lin,Room 101
1,2,Bob,Science,Dr. Wang,Room 202
2,3,Charlie,Math,Dr. Lin,Room 101
3,1,Alice,English,Dr. Smith,Room 303
4,2,Bob,English,Dr. Smith,Room 303
